In [1]:
import os 
os.chdir("../")
%pwd

'd:\\Programming\\ML\\End-to-End\\CV\\End-to-End-PlantVillage'

In [2]:
from src.PlantVillage import logger 
from src.PlantVillage.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.PlantVillage.utils import create_directories, read_yaml 

from dataclasses import dataclass
from pathlib import Path
import os 
import kagglehub
from ensure import ensure_annotations


In [3]:
@dataclass(frozen=True)
class DataIngestionSchema:
    root_dir: Path 
    source_url: str
    local_path: str
    

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionSchema:
        config = self.config.data_ingestion 
        
        create_directories([config.root_dir])
        
        return DataIngestionSchema(
            root_dir= config.root_dir, 
            source_url= config.source_url,
            local_path= config.local_path
        )
        
        
        

In [5]:
class DataIngestion:
    def __init__(self, config: DataIngestionSchema):
        self.config = config
        
    @ensure_annotations
    def download_data(self):
        try:
            # download data from kagglehub
            data_url = self.config.source_url
            if "datasets" in data_url:
                data_url = data_url.split("datasets/")[-1]
                
            # custom download path
            os.environ["KAGGLEHUB_CACHE"] = self.config.local_path
            # start download
            kagglehub.dataset_download(data_url)
            
            # copy csv to target path (local_path)
            target_path = os.path.join(self.config.local_path) 
            

        except Exception as e:
            raise ValueError(f"Invalid URL: must be user/dataset-name ERROR: {e}")

In [6]:
try:
    config = ConfigurationManager() 
    data_ingestion_config = config.get_data_ingestion_config() 
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
except Exception as e:
    raise e

[2026-01-25 19:55:40,977][INFO][__init__] -> yaml file: config\config.yaml loaded Successfully.
[2026-01-25 19:55:40,979][INFO][__init__] -> yaml file: params.yaml loaded Successfully.
[2026-01-25 19:55:40,981][INFO][__init__] -> Directory created Successfully at: artifacts
[2026-01-25 19:55:40,982][INFO][__init__] -> Directory created Successfully at: artifacts/data_ingestion


100%|██████████| 6.98k/6.98k [00:00<00:00, 7.14MB/s]

Extracting files...
